## Set Date Range

In [8]:
start_dt = '2025-11-01'
end_dt = '2025-11-07'

## Invited Users - Batch 1

In [9]:
# invited = pd.read_csv("dw_invited_users.csv") # This is local csv file
invited_users = 'sparky_panel_updated' # This is how data is saved in hive and BQ

## Load Libraries

In [10]:
import os
import pickle
from datetime import datetime, timedelta
from io import BytesIO
from trino.dbapi import connect
from trino.auth import BasicAuthentication

import pandas as pd
import db_dtypes
import pytz
#from src.config import SETTINGS
from google.auth.exceptions import DefaultCredentialsError
from google.cloud import bigquery#, storage
import warnings

warnings.filterwarnings("ignore")
from dotenv import load_dotenv
load_dotenv()

True

## Setting up BQ and Hive connection

In [11]:
host_name = "presto-datadiscovery.walmart.com"
port_number = 8443
user_name = "k0t0e62"
password = os.getenv("DB_PASSWORD")

In [12]:
conn = connect(
    host=host_name,
    port=port_number,
    user=user_name,
    auth=BasicAuthentication(user_name, password),
    catalog="hive",
    http_scheme="https",
    verify='./merged_cacert.pem'
)

cur = conn.cursor()

In [13]:
def get_bigquery_client(project: str) -> bigquery.Client:
    
    if not os.environ["GOOGLE_APPLICATION_CREDENTIALS"]:
        raise DefaultCredentialsError(
            """GOOGLE_APPLICATION_CREDENTIALS environment variable not set. 
                    Please go to .env file and set it to the path of your GCP service account key file."""
        )

    if not project:
        raise EnvironmentError(
            "GCP_PROJECT environment variable not set. Please go to .env file and set it to your GCP project ID."
        )

    return bigquery.Client(project=os.environ["GCP_PROJECT"])


# ----------------------- Load BigQuery Data -----------------------
def load_bq_data(query: str) -> pd.DataFrame:
    """
    Load data from BigQuery.

    Parameters:
    - query: SQL query to execute
    - client: BigQuery client

    Returns:
    - Pandas DataFrame containing the query result
    """

    client = get_bigquery_client(project=os.environ["GCP_PROJECT"])
    return client.query(query).to_dataframe()


# ----------------------- Load BigQuery Data -----------------------
def write_dataframe_to_bigquery(
    dataframe: pd.DataFrame,
    table_name: str,
    schema=None,
    drop_table=True,
) -> None:
    """
    Write a pandas DataFrame to a BigQuery table.
    ! by default, it drops wmt-transplan-ib-forecasting.transplan_ob_forecasting.{table_name} table
    ! please refactor in the future release to make it more generic

    Parameters:
    - dataframe: Pandas DataFrame to write to BigQuery
    - table_name: Name of the BigQuery table to write to
    - schema: Schema of data structure
    - bq_client: BigQuery client object

    Returns:
    - BigQuery job result
    """
    bq_client = get_bigquery_client(project=os.environ["GCP_PROJECT"])

    if drop_table:
        bq_client.query(
            f"DROP TABLE IF EXISTS wmt-transplan-ib-forecasting.transplan_ib_forecasting.{table_name}"
        )
    target_dataset = bq_client.dataset("transplan_ib_forecasting")
    target_table = target_dataset.table(table_name)
    job = bq_client.load_table_from_dataframe(
        dataframe,
        target_table,
        job_config=schema,
    )
    return job.result()

# Customer Usage Metric 0 - Total classic app sessions

In [ ]:
print("Total classic app sessions")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})


SELECT COUNT(DISTINCT a.pg_cust_id) classic_users, 
       COUNT(DISTINCT a.brwsr_sessn_cooke_val) classic_sessn
FROM WW_CSD_DL_TABLES.CSD_SITE_TRAFFIC_EVENT_LOG_VW a
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND TENANT_STE_CD In ('US_GLASS_ANDROID', 'US_GLASS_IOS')
AND actn_catg_nm = 'pageView'
AND BOT_IND = 0
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
glass_app_session = pd.DataFrame(rows, columns=cols)
glass_app_session.head()



Total classic app sessions


,classic_users,classic_sessn
0,12240,108805


## Customer Usage Engagement Funnel Metric 1 - Logged In Users/ Sessions

In [39]:
print("Logged in users/ sessions")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})

SELECT COUNT(DISTINCT a.pg_cust_id) AS login_users,
       COUNT(DISTINCT a.brwsr_sessn_cooke_val) AS login_sessn
FROM (SELECT DISTINCT pg_cust_id, visitor_cooke_val, brwsr_sessn_cooke_val
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND json_extract_scalar(page_attr, '$.nm') = 'homePage' -- Users that reached home page
AND actn_catg_nm = 'pageView') a
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
login_users = pd.DataFrame(rows, columns=cols)
login_users.head()

Logged in users/ sessions


,login_users,login_sessn
0,7881,42897


## Customer Usage Engagement Funnel Metric 2 - Interacted Users

In [40]:
print("Interacted Users")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})

SELECT
  COUNT(DISTINCT a.pg_cust_id) AS interacted_users
FROM (SELECT DISTINCT pg_cust_id FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND actn_catg_nm IN ('onClick') 
AND pg_cust_id IS NOT NULL) a -- This will exclude the onboarding events
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
interacted_users = pd.DataFrame(rows, columns=cols)
interacted_users.head()

Interacted Users


,interacted_users
0,7859


## Customer Usage Engagement Funnel Metric 2 - Interacted Sessions

In [41]:
print("Interacted Sessions")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})

SELECT
  COUNT(DISTINCT brwsr_sessn_cooke_val) AS interacted_sessn
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw a 
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND actn_catg_nm IN ('onClick') 
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
interacted_sessn = pd.DataFrame(rows, columns=cols)
interacted_sessn.head()

Interacted Sessions


,interacted_sessn
0,40128


## Metric 3- Users with at least 1 bounce session/ Bounce Sessions

In [42]:
print("Bounced Users/ Sessions")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users}),

bounce_sessions AS (
SELECT
  pg_cust_id,
  brwsr_sessn_cooke_val
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
  AND (assoc_ind = 'false' OR assoc_ind IS NULL)
  AND actn_catg_nm IN ('onClick','pageView')
  AND pg_cust_id IS NOT NULL
GROUP BY pg_cust_id, brwsr_sessn_cooke_val
HAVING
  -- no clicks in the session
  SUM(CASE WHEN actn_catg_nm = 'onClick' THEN 1 ELSE 0 END) = 0
  -- at least one homepage pageView
  AND SUM(CASE WHEN actn_catg_nm = 'pageView' AND cntxt_nm = 'homePage' THEN 1 ELSE 0 END) >= 1
  -- no pageViews that are NOT homepage
  AND SUM(CASE WHEN actn_catg_nm = 'pageView' AND cntxt_nm <> 'homePage' THEN 1 ELSE 0 END) = 0)


SELECT COUNT(DISTINCT a.pg_cust_id) bounce_cust_ct,
       COUNT(DISTINCT a.brwsr_sessn_cooke_val) bounce_sessn_ct
FROM bounce_sessions a
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
bounce_users = pd.DataFrame(rows, columns=cols)
bounce_users.head()

Bounced Users/ Sessions


,bounce_cust_ct,bounce_sessn_ct
0,166,174


## Customer usage Engagement Funnel Metric 
 1. Meaningfully Interacted Users, Session, Count
 2. Users, Sessions, Count with message
 3. Users, Sessions, Count of Click category tile
 4. Users, Sessions, Count of Click Home Page tile
 5. Users, Sessions, Count of Click Recent tile

In [43]:
print("Meaningfully Interacted Users")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users}),

meaningful_interactions AS (SELECT
  pg_cust_id,
  MAX(CASE WHEN (json_extract_scalar(payload_txt, '$.eventType') = 'SPARKY_CHAT_SEND') THEN 1 ELSE 0 END) AS sent_msg,
  MAX(CASE WHEN (cntxt_nm = 'recentPage' AND json_extract_scalar(payload_txt, '$.moduleName') IN ('OrderCard', 'ChatCard', 'ShopCard')) THEN 1 ELSE 0 END) AS c_recent_card,
  MAX(CASE WHEN (cntxt_nm = 'homePage' AND json_extract_scalar(payload_txt, '$.moduleName') IN ('SHOP', 'WELCOME', 'PRODUCT')) THEN 1 ELSE 0 END) AS c_home_card,
  MAX(CASE WHEN (cntxt_nm = 'categoryPage' AND json_extract_scalar(payload_txt, '$.moduleType') = 'categoryCard') THEN 1 ELSE 0 END) AS c_category_card
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND ((json_extract_scalar(payload_txt, '$.eventType') = 'SPARKY_CHAT_SEND') -- Sent a message on Sparky
    OR (cntxt_nm = 'recentPage' 
          AND actn_catg_nm = 'onClick' 
          AND json_extract_scalar(payload_txt, '$.actionSubCateg') = 'cardClick' 
          AND json_extract_scalar(payload_txt, '$.moduleName') IN ('OrderCard', 'ChatCard', 'ShopCard')
      ) -- Clicked on Recent cards
    OR (cntxt_nm = 'homePage' 
          AND actn_catg_nm = 'onClick' 
          AND json_extract_scalar(payload_txt, '$.actionSubCateg') = 'cardClick' 
          AND json_extract_scalar(payload_txt, '$.moduleName') IN ('SHOP', 'WELCOME', 'PRODUCT')
      ) -- Clicked on Home Page Cards
    OR (cntxt_nm = 'categoryPage'
          AND actn_catg_nm = 'onClick' 
          AND json_extract_scalar(payload_txt, '$.actionSubCateg') = 'cardClick' 
          AND json_extract_scalar(payload_txt, '$.moduleType') = 'categoryCard'
      ) --  Clicked on Category Cards
    )
GROUP BY 1
)


SELECT COUNT(DISTINCT a.pg_cust_id) AS users_meaningful_interaction,
       COUNT(DISTINCT CASE WHEN sent_msg = 1 THEN a.pg_cust_id END) AS users_sent_msg,
       COUNT(DISTINCT CASE WHEN c_recent_card = 1 THEN a.pg_cust_id END) AS users_click_recent_card,
       COUNT(DISTINCT CASE WHEN c_home_card = 1 THEN a.pg_cust_id END) AS users_click_home_card,
       COUNT(DISTINCT CASE WHEN c_category_card = 1 THEN a.pg_cust_id END) AS users_click_category_card
FROM meaningful_interactions a
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users

"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
meaningful_interacted_users = pd.DataFrame(rows, columns=cols)
meaningful_interacted_users.head()

Meaningfully Interacted Users


,users_meaningful_interaction,users_sent_msg,users_click_recent_card,users_click_home_card,users_click_category_card
0,7272,6543,3268,4460,5243


In [44]:
print("Meaningfully Interacted Sessions")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users}),

meaningful_interactions AS (SELECT a.pg_cust_id,
  brwsr_sessn_cooke_val,
  MAX(CASE WHEN (json_extract_scalar(payload_txt, '$.eventType') = 'SPARKY_CHAT_SEND') THEN 1 ELSE 0 END) AS sent_msg,
  MAX(CASE WHEN (cntxt_nm = 'recentPage' AND json_extract_scalar(payload_txt, '$.moduleName') IN ('OrderCard', 'ChatCard', 'ShopCard')) THEN 1 ELSE 0 END) AS c_recent_card,
  MAX(CASE WHEN (cntxt_nm = 'homePage' AND json_extract_scalar(payload_txt, '$.moduleName') IN ('SHOP', 'WELCOME', 'PRODUCT')) THEN 1 ELSE 0 END) AS c_home_card,
  MAX(CASE WHEN (cntxt_nm = 'categoryPage' AND json_extract_scalar(payload_txt, '$.moduleType') = 'categoryCard') THEN 1 ELSE 0 END) AS c_category_card
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw a 
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND ((json_extract_scalar(payload_txt, '$.eventType') = 'SPARKY_CHAT_SEND') -- Sent a message on Sparky
    OR (cntxt_nm = 'recentPage' 
          AND actn_catg_nm = 'onClick' 
          AND json_extract_scalar(payload_txt, '$.actionSubCateg') = 'cardClick' 
          AND json_extract_scalar(payload_txt, '$.moduleName') IN ('OrderCard', 'ChatCard', 'ShopCard')
      ) -- Clicked on Recent cards
    OR (cntxt_nm = 'homePage' 
          AND actn_catg_nm = 'onClick' 
          AND json_extract_scalar(payload_txt, '$.actionSubCateg') = 'cardClick' 
          AND json_extract_scalar(payload_txt, '$.moduleName') IN ('SHOP', 'WELCOME', 'PRODUCT')
      ) -- Clicked on Home Page Cards
    OR (cntxt_nm = 'categoryPage'
          AND actn_catg_nm = 'onClick' 
          AND json_extract_scalar(payload_txt, '$.actionSubCateg') = 'cardClick' 
          AND json_extract_scalar(payload_txt, '$.moduleType') = 'categoryCard'
      ) --  Clicked on Category Cards
    )
GROUP BY 1,2
)


SELECT 
      COUNT(DISTINCT brwsr_sessn_cooke_val) AS sessn_meaningful_interaction,
      COUNT(DISTINCT CASE WHEN sent_msg = 1 THEN brwsr_sessn_cooke_val END) AS sessn_sent_msg,
      COUNT(DISTINCT CASE WHEN c_recent_card = 1 THEN brwsr_sessn_cooke_val END) AS sessn_click_recent_card,
      COUNT(DISTINCT CASE WHEN c_home_card = 1 THEN brwsr_sessn_cooke_val END) AS sessn_click_home_card,
      COUNT(DISTINCT CASE WHEN c_category_card = 1 THEN brwsr_sessn_cooke_val END) AS sessn_click_category_card
FROM meaningful_interactions
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
meaningful_interacted_sessn = pd.DataFrame(rows, columns=cols)
meaningful_interacted_sessn.head()

Meaningfully Interacted Sessions


,sessn_meaningful_interaction,sessn_sent_msg,sessn_click_recent_card,sessn_click_home_card,sessn_click_category_card
0,31006,21768,6277,7855,12103


In [45]:
print("Event Counts for Each Interaction Type")

sql = f"""
WITH invited_users AS (
    SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
    FROM gpadata_analytics.{invited_users}
)
SELECT

    -- ALL meaningful interaction events (union of all above, counted at the event/log level, not user)
    (SELECT COUNT(*)
        FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
        JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
        WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
          AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
          AND (
                json_extract_scalar(e.payload_txt, '$.eventType') = 'SPARKY_CHAT_SEND'
                OR (
                    e.cntxt_nm = 'recentPage'
                    AND e.actn_catg_nm = 'onClick'
                    AND json_extract_scalar(e.payload_txt, '$.actionSubCateg') = 'cardClick'
                    AND json_extract_scalar(e.payload_txt, '$.moduleName') IN ('OrderCard', 'ChatCard', 'ShopCard')
                )
                OR (
                    e.cntxt_nm = 'homePage'
                    AND e.actn_catg_nm = 'onClick'
                    AND json_extract_scalar(e.payload_txt, '$.actionSubCateg') = 'cardClick'
                    AND json_extract_scalar(e.payload_txt, '$.moduleName') IN ('SHOP', 'WELCOME', 'PRODUCT')
                )
                OR (
                    e.cntxt_nm = 'categoryPage'
                    AND e.actn_catg_nm = 'onClick'
                    AND json_extract_scalar(e.payload_txt, '$.actionSubCateg') = 'cardClick'
                    AND json_extract_scalar(e.payload_txt, '$.moduleType') = 'categoryCard'
                )
          )
    ) AS count_meaningful_interaction,

    -- Sent message count
    (SELECT COUNT(*)
        FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
        JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
        WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
          AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
          AND json_extract_scalar(e.payload_txt, '$.eventType') = 'SPARKY_CHAT_SEND'
    ) AS count_sent_msg,

    -- Clicked recent card count
    (SELECT COUNT(*)
        FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
        JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
        WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
          AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
          AND e.cntxt_nm = 'recentPage'
          AND e.actn_catg_nm = 'onClick'
          AND json_extract_scalar(e.payload_txt, '$.actionSubCateg') = 'cardClick'
          AND json_extract_scalar(e.payload_txt, '$.moduleName') IN ('OrderCard', 'ChatCard', 'ShopCard')
    ) AS count_recent_card,

    -- Clicked home card count
    (SELECT COUNT(*)
        FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
        JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
        WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
          AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
          AND e.cntxt_nm = 'homePage'
          AND e.actn_catg_nm = 'onClick'
          AND json_extract_scalar(e.payload_txt, '$.actionSubCateg') = 'cardClick'
          AND json_extract_scalar(e.payload_txt, '$.moduleName') IN ('SHOP', 'WELCOME', 'PRODUCT')
    ) AS count_home_card,

    -- Clicked category card count
    (SELECT COUNT(*)
        FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
        JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
        WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
          AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
          AND e.cntxt_nm = 'categoryPage'
          AND e.actn_catg_nm = 'onClick'
          AND json_extract_scalar(e.payload_txt, '$.actionSubCateg') = 'cardClick'
          AND json_extract_scalar(e.payload_txt, '$.moduleType') = 'categoryCard'
    ) AS count_category_card

    
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]
meaningful_interected_event_counts = pd.DataFrame(rows, columns=cols)
meaningful_interected_event_counts.head()

Event Counts for Each Interaction Type


,count_meaningful_interaction,count_sent_msg,count_recent_card,count_home_card,count_category_card
0,99467,56946,10205,11418,20898


## Customer usage Engagement Funnel Metric 
 1. Users, Session, Count who open shop
 2. Users, Sessions, Count who open category shop
 3. Users, Sessions, Count who open weekly shop
 4. Users, Sessions, Count who open browse shop
 5. Users, Sessions, Count who open Planning (events) shop 

In [46]:
print("Users that opened a shop")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})

SELECT
  COUNT(DISTINCT a.pg_cust_id) AS cust_opened_shop,
  COUNT(DISTINCT CASE WHEN cntxt_nm = 'eventShop' THEN a.pg_cust_id END) AS cust_opened_event_shop,
  COUNT(DISTINCT CASE WHEN cntxt_nm = 'weeklyShop' THEN a.pg_cust_id END) AS cust_opened_weekly_shop,
  COUNT(DISTINCT CASE WHEN cntxt_nm = 'browseShop' THEN a.pg_cust_id END) AS cust_opened_browse_shop,
  COUNT(DISTINCT CASE WHEN cntxt_nm = 'categoryShop' THEN a.pg_cust_id END) AS cust_opened_category_shop
FROM (SELECT DISTINCT pg_cust_id, cntxt_nm FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND actn_catg_nm = 'pageView'
AND cntxt_nm in ('eventShop', 'weeklyShop', 'browseShop', 'categoryShop')) a
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
opened_shop_cust = pd.DataFrame(rows, columns=cols)
opened_shop_cust.head()

Users that opened a shop


,cust_opened_shop,cust_opened_event_shop,cust_opened_weekly_shop,cust_opened_browse_shop,cust_opened_category_shop
0,5859,475,2212,258,5362


In [47]:
print("Sessions in which a shop was open")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})

SELECT
  COUNT(DISTINCT a.brwsr_sessn_cooke_val) AS sessn_opened_shop,
  COUNT(DISTINCT CASE WHEN cntxt_nm = 'eventShop' THEN a.brwsr_sessn_cooke_val END) AS sessn_opened_event_shop,
  COUNT(DISTINCT CASE WHEN cntxt_nm = 'weeklyShop' THEN a.brwsr_sessn_cooke_val END) AS sessn_opened_weekly_shop,
  COUNT(DISTINCT CASE WHEN cntxt_nm = 'browseShop' THEN a.brwsr_sessn_cooke_val END) AS sessn_opened_browse_shop,
  COUNT(DISTINCT CASE WHEN cntxt_nm = 'categoryShop' THEN a.brwsr_sessn_cooke_val END) AS sessn_opened_category_shop
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw a 
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND actn_catg_nm = 'pageView'
AND cntxt_nm in ('eventShop', 'weeklyShop', 'browseShop', 'categoryShop')
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
opened_shop_sessn = pd.DataFrame(rows, columns=cols)
opened_shop_sessn.head()

Sessions in which a shop was open


,sessn_opened_shop,sessn_opened_event_shop,sessn_opened_weekly_shop,sessn_opened_browse_shop,sessn_opened_category_shop
0,15008,610,3309,315,12827


In [48]:
print("Event Counts for Shop Opens")

sql = f"""
WITH invited_users AS (
    SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
    FROM gpadata_analytics.{invited_users}
)
SELECT
    -- Total events opening any shop context
    (SELECT COUNT(*)
        FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
        JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
        WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
          AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
          AND e.actn_catg_nm = 'pageView'
          AND e.cntxt_nm IN ('eventShop', 'weeklyShop', 'browseShop', 'categoryShop')
    ) AS count_opened_shop,

    -- Events opening eventShop
    (SELECT COUNT(*)
        FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
        JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
        WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
          AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
          AND e.actn_catg_nm = 'pageView'
          AND e.cntxt_nm = 'eventShop'
    ) AS count_opened_event_shop,

    -- Events opening weeklyShop
    (SELECT COUNT(*)
        FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
        JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
        WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
          AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
          AND e.actn_catg_nm = 'pageView'
          AND e.cntxt_nm = 'weeklyShop'
    ) AS count_opened_weekly_shop,

    -- Events opening browseShop
    (SELECT COUNT(*)
        FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
        JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
        WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
          AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
          AND e.actn_catg_nm = 'pageView'
          AND e.cntxt_nm = 'browseShop'
    ) AS count_opened_browse_shop,

    -- Events opening categoryShop
    (SELECT COUNT(*)
        FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
        JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
        WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
          AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
          AND e.actn_catg_nm = 'pageView'
          AND e.cntxt_nm = 'categoryShop'
    ) AS count_opened_category_shop
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]
count_opened_shop = pd.DataFrame(rows, columns=cols)
count_opened_shop.head()


Event Counts for Shop Opens


,count_opened_shop,count_opened_event_shop,count_opened_weekly_shop,count_opened_browse_shop,count_opened_category_shop
0,44874,1197,7357,635,35685


## Customer usage Engagement Funnel Metric 
 1. Users, Count that created shop
 2. Users, Count that created browse shop
 3. Users, Count that created Planning (events) shop 

In [49]:
print("Users that created a shop")

sql = f"""
WITH invited_users AS (
    SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
    FROM gpadata_analytics.{invited_users}
)
SELECT
    COUNT(DISTINCT e.pg_cust_id) AS created_shop_users,
    COUNT(DISTINCT e.brwsr_sessn_cooke_val) AS created_shop_sessn,
    COUNT(*) AS count_created_shop
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
  AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
  AND JSON_EXTRACT_SCALAR(e.payload_txt, '$.eventType') = 'SPARKY_CHAT_RECEIVE'
  AND JSON_EXTRACT_SCALAR(e.payload_txt, '$.responseType') = 'SHOP_MESSAGE'
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]
created_shop = pd.DataFrame(rows, columns=cols)
created_shop.head()




Users that created a shop


,created_shop_users,created_shop_sessn,count_created_shop
0,674,802,1013


In [50]:
print("Users that created a shop")

sql = f"""
WITH invited_users AS (
    SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
    FROM gpadata_analytics.{invited_users}
)
SELECT
    COUNT(DISTINCT CASE WHEN json_extract_scalar(payload_txt, '$.shopType') = 'EVENT' THEN e.pg_cust_id END) AS users_created_event_shop,
    COUNT(DISTINCT CASE WHEN json_extract_scalar(payload_txt, '$.shopType') = 'BROWSE' THEN e.pg_cust_id END) AS users_created_browse_shop,
    COUNT(DISTINCT CASE WHEN json_extract_scalar(payload_txt, '$.shopType') = 'EVENT' THEN brwsr_sessn_cooke_val END) AS sessn_created_event_shop,
    COUNT(DISTINCT CASE WHEN json_extract_scalar(payload_txt, '$.shopType') = 'BROWSE' THEN brwsr_sessn_cooke_val END) AS sessn_created_browse_shop,
    COUNT(CASE WHEN json_extract_scalar(payload_txt, '$.shopType') = 'EVENT' THEN 1 END) AS count_created_event_shop,
    COUNT(CASE WHEN json_extract_scalar(payload_txt, '$.shopType') = 'BROWSE' THEN 1 END) AS count_created_browse_shop
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
  AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
  AND JSON_EXTRACT_SCALAR(e.payload_txt, '$.eventType') = 'SPARKY_CHAT_RECEIVE'
  AND JSON_EXTRACT_SCALAR(e.payload_txt, '$.responseType') = 'SHOP_MESSAGE'
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]
created_shop_type = pd.DataFrame(rows, columns=cols)
created_shop_type.head()


Users that created a shop


,users_created_event_shop,users_created_browse_shop,sessn_created_event_shop,sessn_created_browse_shop,count_created_event_shop,count_created_browse_shop
0,341,175,378,191,474,238


## Customer usage Engagement Funnel Metric - Users, Session, Count with ATC / Add All to cart


In [51]:
print("Users that did ATC/ Total ATCs")
sql = f"""
WITH invited_users AS (
    SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
    FROM gpadata_analytics.{invited_users}
),
atc_clicks AS (
    SELECT 
        event_dt,
        pg_cust_id,
        brwsr_sessn_cooke_val,
        actn_sub_catg_nm,
        cntxt_nm,
        CAST(
            from_unixtime(cast(event_ts_epoch as double) / 1000) 
            AT TIME ZONE 'America/Los_Angeles' 
            AS timestamp
        ) AS event_ts_epoch,
        json_extract_scalar(page_attr, '$.nm') AS page_nm,
        json_extract_scalar(payload_txt, '$.r') AS ref_page,
        json_extract_scalar(payload_txt, '$.overlayName') AS overlay_nm,
        COALESCE(
            json_extract_scalar(payload_txt, '$.itemDetails.itemId'),
            json_extract_scalar(payload_txt, '$.itemId'),
            json_extract_scalar(payload_txt, '$.itemDetails.usitemId')
        ) AS item_id,
        MAX(json_extract_scalar(payload_txt, '$.shopId')) AS shop_id
    FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
    WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
      AND (assoc_ind = 'false' OR assoc_ind IS NULL) -- Exclude associates
      AND actn_catg_nm = 'onClick'
      AND actn_sub_catg_nm IN ('addToCart', 'addAllToCart')
      AND json_extract_scalar(page_attr, '$.nm') != 'arPlatform'
    GROUP BY 1,2,3,4,5,6,7,8,9,10
),
dedupe AS (
    SELECT 
        a.*,
        ROW_NUMBER() OVER (
            PARTITION BY a.pg_cust_id, brwsr_sessn_cooke_val, actn_sub_catg_nm, item_id
            ORDER BY shop_id, overlay_nm, event_ts_epoch
        ) AS rn -- Remove duplicates
    FROM atc_clicks a
    JOIN invited_users i ON i.pg_cust_id = a.pg_cust_id
)
SELECT 
    COUNT(DISTINCT a.pg_cust_id) AS cust_w_atc_both,
    COUNT(DISTINCT CASE WHEN actn_sub_catg_nm = 'addAllToCart' THEN a.pg_cust_id END) AS cust_w_aatc,
    COUNT(DISTINCT CASE WHEN actn_sub_catg_nm = 'addToCart' THEN a.pg_cust_id END) AS cust_w_atc,
    COUNT(DISTINCT brwsr_sessn_cooke_val) AS sessn_w_atc_both,
    COUNT(DISTINCT CASE WHEN actn_sub_catg_nm = 'addAllToCart' THEN a.brwsr_sessn_cooke_val END) AS sessn_w_aatc,
    COUNT(DISTINCT CASE WHEN actn_sub_catg_nm = 'addToCart' THEN a.brwsr_sessn_cooke_val END) AS sessn_w_atc,
    COUNT(*) AS total_atc,
    COUNT(CASE WHEN actn_sub_catg_nm = 'addToCart' THEN 1 END) AS count_w_atc,
    COUNT(CASE WHEN actn_sub_catg_nm = 'addAllToCart' THEN 1 END) AS count_w_aatc
FROM dedupe a
WHERE rn = 1
"""
cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]  # extract column names
atc_users = pd.DataFrame(rows, columns=cols)
atc_users.head()


Users that did ATC/ Total ATCs


,cust_w_atc_both,cust_w_aatc,cust_w_atc,sessn_w_atc_both,sessn_w_aatc,sessn_w_atc,total_atc,count_w_atc,count_w_aatc
0,3344,367,3297,7406,427,7246,24814,24387,427


## Users and sessions that clicked on continue to checkout


In [52]:
print("Users that clicked on continue to cart")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})

SELECT
  COUNT(DISTINCT a.pg_cust_id) AS cust_w_checkout
FROM (SELECT DISTINCT pg_cust_id FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND actn_catg_nm IN ('onClick') 
AND actn_sub_catg_nm = 'continueToCheckout'
AND cntxt_nm = 'cart') a -- This will exclude the onboarding events
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users

"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
checkout_users = pd.DataFrame(rows, columns=cols)
checkout_users.head()

Users that clicked on continue to cart


,cust_w_checkout
0,1620


In [53]:
print("Sessions with click on continue to checkout")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})

SELECT
  COUNT(DISTINCT a.brwsr_sessn_cooke_val) AS sessn_w_checkout
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw a 
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND actn_catg_nm IN ('onClick') 
AND actn_sub_catg_nm = 'continueToCheckout'
AND cntxt_nm = 'cart'-- This will exclude the onboarding events
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
checkout_sessn = pd.DataFrame(rows, columns=cols)
checkout_sessn.head()

Sessions with click on continue to checkout


,sessn_w_checkout
0,2491


## Customer Usage Engagement Funnel - Users, Sessions that had an order / Total orders


In [54]:
print("Users that placed an order/ Total orders")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})

SELECT
  COUNT(DISTINCT a.pg_cust_id) AS cust_w_orders,
  COUNT(DISTINCT order_id) as total_orders
FROM (SELECT DISTINCT event_dt,
                 pg_cust_id, 
                 json_extract_scalar(payload_txt, '$.orderId') as order_id
      FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
      WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
      AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
      AND cntxt_nm = 'checkout'
      AND json_extract_scalar(page_attr, '$.nm') = 'thankYou'
      AND actn_catg_nm = 'pageView') a -- This will exclude the onboarding events
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
order_users = pd.DataFrame(rows, columns=cols)
order_users.head()

Users that placed an order/ Total orders


,cust_w_orders,total_orders
0,1174,1750


In [55]:
print("Sessions with orders")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})

SELECT
  COUNT(DISTINCT a.brwsr_sessn_cooke_val) AS sessn_w_orders
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw a 
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND cntxt_nm = 'checkout'
AND json_extract_scalar(page_attr, '$.nm') = 'thankYou'
AND actn_catg_nm = 'pageView' -- This will exclude the onboarding events
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
order_sessn = pd.DataFrame(rows, columns=cols)
order_sessn.head()

Sessions with orders


,sessn_w_orders
0,1702


## Metric - In Chat Bounced Users


In [56]:
print("In Chat Bounced Users")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users}),

clicked_on_sparky AS (
    SELECT DISTINCT 
        pg_cust_id, 
        brwsr_sessn_cooke_val
    FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
    WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
    AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
    AND actn_catg_nm = 'onClick'
    AND actn_sub_catg_nm = 'sparky'
),

sent_msg AS (
    SELECT DISTINCT 
        pg_cust_id, 
        brwsr_sessn_cooke_val
    FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
    WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
    AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
    AND json_extract_scalar(payload_txt, '$.eventType') = 'SPARKY_CHAT_SEND'
)


SELECT COUNT(DISTINCT a.pg_cust_id) AS chat_bounce_users,
       COUNT(DISTINCT a.brwsr_sessn_cooke_val) AS chat_bounce_sessn
FROM clicked_on_sparky a
LEFT JOIN sent_msg b 
ON a.pg_cust_id = b.pg_cust_id AND a.brwsr_sessn_cooke_val = b.brwsr_sessn_cooke_val
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
WHERE b.brwsr_sessn_cooke_val is null
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
inchat_bounce_users = pd.DataFrame(rows, columns=cols)
inchat_bounce_users.head()

In Chat Bounced Users


,chat_bounce_users,chat_bounce_sessn
0,1265,1450


## System Performance - Total Messages, Error, Fallback

In [57]:
print("Total Messages, Error, Fallback")

inchat_msg = load_bq_data(f"""
                   WITH invited_users AS (
                    SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
                    FROM `wmt-et-ccomm-prod.temp.{invited_users}`)

                  SELECT
                  COUNT(DISTINCT correlation_id) AS total_msg,
                  COUNT(DISTINCT CASE WHEN REGEXP_EXTRACT(current_entities, '\\\\{{([^=]+)=' ) = 'fallback_category' THEN correlation_id END) AS fallback_msg,
                  COUNT(DISTINCT CASE WHEN converse_intent is null
                                      OR converse_intent = ""
                                      OR converse_short_reply like "%Sorry, there was a problem while processing your request. Can I help you with anything else?%"
                                      OR converse_short_reply like "%Your user id is already in use. Please try again after sometime.%"
                                      OR converse_short_reply like "%not quite working on my end, please try again later.%"
                                      THEN correlation_id END) AS error_msg
                  FROM `wmt-et-converse-prod.converse_logs_ds_xrsamurai.logs_parsed` a
                  JOIN invited_users i
                  ON i.pg_cust_id = a.customer_id -- To only keep the invited users
                  WHERE DATE(DATETIME(TIMESTAMP(timestamp), 'America/Los_Angeles')) BETWEEN '{start_dt}' AND '{end_dt}'
                  AND channel_id = 'RAPTOR_IOS_APP'
                    """)

inchat_msg.head()


Total Messages, Error, Fallback


,total_msg,fallback_msg,error_msg
0,54452,3067,1394


## System Performace - Latency (p50, p75, p90, p95, p99)

1. Use BQ link - https://console.cloud.google.com/bigquery?ws=!1m7!1m6!12m5!1m3!1swmt-et-ccomm-prod!2sus-central1!3s58ceb105-6316-469a-8911-49056942ef35!2e1

## Interaction Experience - Duration of Session

In [58]:
print("Average, Min, and Max Session Duration (ms and seconds)")

sql = f"""
WITH invited_users AS (
    SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
    FROM gpadata_analytics.{invited_users}
),
session_durations AS (
    SELECT
        brwsr_sessn_cooke_val AS session_id,
        CAST(MIN(CAST(event_ts_epoch AS BIGINT)) AS BIGINT) AS min_ts,
        CAST(MAX(CAST(event_ts_epoch AS BIGINT)) AS BIGINT) AS max_ts,
        MAX(CAST(event_ts_epoch AS BIGINT)) - MIN(CAST(event_ts_epoch AS BIGINT)) AS session_duration_ms
    FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
    JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
    WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
    GROUP BY brwsr_sessn_cooke_val
)
SELECT
    ROUND(AVG(session_duration_ms) / 1000, 2) AS avg_session_duration_sec,
    ROUND(MIN(session_duration_ms) / 1000, 2) AS min_session_duration_sec,
    ROUND(MAX(session_duration_ms) / 1000, 2) AS max_session_duration_sec
FROM session_durations
WHERE session_duration_ms > 0
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]
session_duration = pd.DataFrame(rows, columns=cols)
session_duration.head()


Average, Min, and Max Session Duration (ms and seconds)


,avg_session_duration_sec,min_session_duration_sec,max_session_duration_sec
0,342.52,0,52594


## Users with Carousel Impressions and Total Item Impressions

In [59]:
print("Users with item carousel impressions")


users_w_carousel_imp = load_bq_data(f"""
WITH invited_users AS (
 SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
 FROM `wmt-et-ccomm-prod.temp.{invited_users}`),

carousel AS (SELECT 
*,
REGEXP_EXTRACT(current_entities, r'products=\[([\s\S]*?)\]') AS products_inner
FROM `wmt-et-converse-prod.converse_logs_ds_xrsamurai.logs_parsed` a
JOIN invited_users i
ON i.pg_cust_id = a.customer_id -- To only keep the invited users
WHERE DATE(DATETIME(TIMESTAMP(timestamp), 'America/Los_Angeles')) BETWEEN '{start_dt}' AND '{end_dt}'
AND channel_id = 'RAPTOR_IOS_APP'),

carousel_item AS (SELECT
 customer_id,
 correlation_id,
 ARRAY_AGG(JSON_EXTRACT_SCALAR(product, '$.usItemId')) AS usItemIds,
 ARRAY_LENGTH(ARRAY_AGG(JSON_EXTRACT_SCALAR(product, '$.usItemId'))) AS usItemCount
FROM carousel,
UNNEST(JSON_EXTRACT_ARRAY(CONCAT('[', products_inner, ']'), '$')) AS product
WHERE products_inner IS NOT NULL
GROUP BY 1, 2)

SELECT COUNT(DISTINCT customer_id) users_w_carousel_imp, SUM(usItemCount) total_item_imp
FROM carousel_item
""")

users_w_carousel_imp.head()

Users with item carousel impressions


,users_w_carousel_imp,total_item_imp
0,5790,169223


## Customer Usage Shop Engagement Reach - Total Carousel Impressions

1. Use Big Query - https://console.cloud.google.com/bigquery?ws=!1m7!1m6!12m5!1m3!1swmt-et-ccomm-prod!2sus-central1!3s030efad7-e863-4aab-b6d9-d0ba613a5f92!2e1

In [60]:
# Merge side by side
df_final = pd.concat([glass_app_session, login_users, interacted_users, interacted_sessn, bounce_users, meaningful_interacted_users, meaningful_interacted_sessn, meaningful_interected_event_counts, 
                      opened_shop_cust, opened_shop_sessn,count_opened_shop, created_shop, created_shop_type, atc_users, checkout_users,
                      checkout_sessn, order_users, order_sessn, inchat_bounce_users, inchat_msg, session_duration, users_w_carousel_imp], axis=1)

df_final

,classic_users,classic_sessn,login_users,login_sessn,interacted_users,interacted_sessn,bounce_cust_ct,bounce_sessn_ct,users_meaningful_interaction,users_sent_msg,...,chat_bounce_users,chat_bounce_sessn,total_msg,fallback_msg,error_msg,avg_session_duration_sec,min_session_duration_sec,max_session_duration_sec,users_w_carousel_imp,total_item_imp
0,14298,468227,7881,42897,7859,40128,166,174,7272,6543,...,1265,1450,54452,3067,1394,342.52,0,52594,5790,169223


In [62]:
df_final.to_clipboard(index=True)

